**Insights**
* There's huge value in having a single file where you can see everything, whether it's a diagram, makefile or Jupyter notebook
* Need to identify what needs to be flexible. Passing a trivial amount of data e2e should be very doable
* Unit tests waste too much time on the edge cases. At the very least, the top test should always be 'happy path' so you can see what it _should_ look like. Then there needs to be a connection between files
    * This is much more possible for data pipelines than application code, so it's under-developed

In [ ]:
DATADIR = ''

In [ ]:
# download_data.py

# loading_dock: [local] sandbox_data.txt, [internet] http server
# processing: download, assign IDs, split out test
# dial: progress bar
# shipping_dock: [local] condensed filetype with out-of-memory append and read
manifest_filename = ''
server_url = ''
output_file_base = ''


In [ ]:
# preprocess_data.py

# loading_dock: [local] train data
# processing: filter, split, tag, format labels
# lever: training, inference, evaluation
# dial: Dask status
# shipping_dock: [local] text-only and label-only files with IDs
status = 'training'
train_filename = ''
filter_bools = {}
split_chars = ()
tag_patterns = {}
output_file = ''


In [2]:
# train_w2v.py

# loading_dock: [local] train text
# processing: gensim w2v
# dial: estimate based on train size
# shipping_dock: [local] gensim model
train_filename = ''
output_file = ''


In [ ]:
# index_text.py

# loading_dock: [local] train text, [local] gensim
# shipping_dock: [local] text as indexes, [local] 100d w2v array sorted with that index
w2v_file = ''
text_file = ''

In [ ]:
# model.py

# loading_dock: 100d w2v array
# lever: params
# dial: model.summary()
# shipping_dock: [local] compiled model



In [ ]:
# train.py

# loading_dock: [local] text as indexes, [local] compiled model
# lever: epocs
# dial: tensorboard
# shipping_dock: [local] saved model

In [ ]:
# inference.py

# loading_dock: link to HN thread
# processing: call preprocess and index, then apply model
# shipping_dock: best comment

In [ ]:
# test.py

# loading_dock: [local] test file
# processing: call inference, then compare to labels 
# shipping_dock: accuracy printout